The notebook will be used for the capstone project.

# PART 1

In [3]:
import csv
import os
import platform
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
output_name = 'toronto'

In [4]:
resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'lxml')
table_classes = {"class": ["sortable", "plainrowheaders"]}
wikitables = soup.findAll("table", table_classes, limit = 1)

In [7]:
def scrape(url, output_name):   
    try:
        os.mkdir(output_name)
    except Exception:  
        pass
    for index, table in enumerate(wikitables):
        if index == 0:
            filename = output_name
        else:
            filename = output_name + '_' + str(index)
        filepath = os.path.join(output_name, filename) + '.csv'
        with open(filepath, mode='w', newline='', encoding='utf-8') as output:
            # Deal with Windows inserting an extra '\r' in line terminators
            if platform.system() == 'Windows':
                kwargs = {'lineterminator': '\n'}
                csv_writer = csv.writer(output,
                                        quoting=csv.QUOTE_ALL,
                                        **kwargs)
            else:
                csv_writer = csv.writer(output,
                                        quoting=csv.QUOTE_ALL)

            write_html_table_to_csv(table, csv_writer)

def write_html_table_to_csv(table, writer):
    """Write HTML table from Wikipedia to a CSV file.
    ARGS:
        table (bs4.Tag): The bs4 Tag object being analyzed.
        writer (csv.writer): The csv Writer object creating the output.
    """
    saved_rowspans = []
    for row in table.findAll("tr"):
        cells = row.findAll(["th", "td"])
        if len(saved_rowspans) == 0:
            saved_rowspans = [None for _ in cells]
        elif len(cells) != len(saved_rowspans):
            for index, rowspan_data in enumerate(saved_rowspans):
                if rowspan_data is not None:
                    value = rowspan_data['value']
                    cells.insert(index, value)
                    if saved_rowspans[index]['rows_left'] == 1:
                        saved_rowspans[index] = None
                    else:
                        saved_rowspans[index]['rows_left'] -= 1
        for index, cell in enumerate(cells):
            if cell.has_attr("rowspan"):
                rowspan_data = {
                    'rows_left': int(cell["rowspan"]),
                    'value': cell,
                }
                saved_rowspans[index] = rowspan_data
        if cells:
            cleaned = clean_data(cells)
            columns_missing = len(saved_rowspans) - len(cleaned)
            if columns_missing:
                cleaned += [None] * columns_missing
            writer.writerow(cleaned)

def clean_data(row):
    """Clean table row list from Wikipedia into a string for CSV.
    ARGS:
        row (bs4.ResultSet): The bs4 result set being cleaned for output.
    RETURNS:
        cleaned_cells (list[str]): List of cleaned text items in this row.
    """
    cleaned_cells = []
    for cell in row:
        references = cell.findAll("sup", {"class": "reference"})
        if references:
            for ref in references:
                ref.extract()
        sortkeys = cell.findAll("span", {"class": "sortkey"})
        if sortkeys:
            for ref in sortkeys:
                ref.extract()
        text_items = cell.findAll(text=True)
        no_footnotes = [text for text in text_items if text[0] != '[']
        cleaned = (
            ''.join(no_footnotes)  
            .replace('\xa0', ' ') 
            .replace('\n', ' ')  
            .strip()
        )
        cleaned_cells += [cleaned]
    return cleaned_cells

scrape(url, output_name)

In [8]:
df = pd.read_csv('toronto/toronto.csv')

In [9]:
df1 = df[df.Borough != 'Not assigned']
df1.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [10]:
df1 = df1.groupby(['Postal Code','Borough'])['Neighborhood'].apply(", ".join).reset_index()
df1.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df1['Neighborhood'] = np.where(df1['Neighborhood'] == 'Not assigned', df1['Borough'], df1['Neighborhood'])
df1

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [12]:
df1.shape

(103, 3)

# PART 2

In [13]:

df2 = pd.read_csv('http://cocl.us/Geospatial_data')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
left = df1
right = df2
result = pd.merge(left, right, on='Postal Code')
result

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
